In [1]:
pip install kafka-python

  Obtaining dependency information for kafka-python from https://files.pythonhosted.org/packages/75/68/dcb0db055309f680ab2931a3eeb22d865604b638acf8c914bedf4c1a0c8c/kafka_python-2.0.2-py2.py3-none-any.whl.metadata
Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from kafka import KafkaProducer, KafkaConsumer
from time import sleep
from json import dumps
import json

In [3]:
 producer = KafkaProducer(bootstrap_servers=['18.144.34.135:9092'], 
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [6]:
producer.send('demo_testing2',value="{'name' : 'VaidehiP'}")